In [1]:
# import libraries
import numpy as np
from numpy import sin, cos, pi as π
from scipy.special import struve, jv
from collections import namedtuple

from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import CustomJS, Slider, Range1d, Span, Toggle, Spinner, RadioButtonGroup
from bokeh.layouts import column, row
output_notebook()

Loading BokehJS ...

In [2]:
# style setup
colors = ['#000000', '#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7']

def set_fig_style(fig,xmax,ymax,xlab='x',ylab='y',xmin=0,ymin=None):
    if ymin is None:
        ymin = -ymax
    for (ax, lab) in [(fig.xaxis,xlab),(fig.yaxis,ylab)]:
        ax.axis_label = lab
        ax.axis_label_text_font_size = '14pt'
        ax.axis_label_text_font_style = 'normal'
        ax.major_label_text_font_size = '12pt'
    fig.legend.label_text_font_size = '14pt'
    fig.x_range = Range1d(xmin,xmax)
    fig.y_range = Range1d(ymin,ymax)

In [3]:
# prepare collection of functions and their Fourier series coefficients
Fourier = namedtuple("Fourier","label, legend, f, a, c, nmax")

def make_fouriers():
    w = 1/3
    adivs = np.r_[0.7099831680351201,0,-0.35445237102618316,0,0.23634150866916137,0,-0.18524301762932088,0,0.1527272615276148,0,-0.13174703549268307,0,0.1160225398530973,0,
                  -0.10434848471309632,0,0.09490903432373918,0,
                  -0.08737631324606303,0,0.08101143183693457,0,-0.07570334975747416,0,0.071087200645023,0,-0.06712140060130616,0,0.0636017469002902,0,-0.06051260268951583,0,0.05772920194201337,0,-0.05524654036328081,0,0.05298327325299655,0,
                  -0.05093896277505697,0,0.049057855773352296,0,-0.04734147588577538,0,0.04575006442607621,0,-0.04428592692728665,0,0.04291978223687456,0,-0.04165416823228488,0,0.040466936827138394,0,-0.03936060192824594,0,0.03831803021689205,
                  0,-0.03734159674381802,0,0.03641779101895347,0,-0.03554880395322289,0,0.03472380414383164,0,-0.033944787416264464,0,0.03320294409076672,0,-0.03250008052465193,0,0.03182893998546995,0,-0.031191152440160776,0,
                  0.030580672838532583,0,-0.02999897061774942,0,0.02944095938666137,0,-0.028907964323224365,0,0.028395667012531437,0,-0.02790526389761672,0,0.027433058719452084,0,-0.02698013345280617,0,0.026543300999415168,0,
                  -0.02612354170283563,0,0.025718089011662094,0,-0.025327833047934237,0,0.024950357551351943,0,-0.024586472491516728,0,0.024234055639737552,0,-0.02389384566433154,0,0.023563968901950748,0,-0.023245100485629406,0,
                  0.022935578262135704,0,-0.02263602063962676,0,0.02234494653818378,0,-0.022062923615893112,0,0.021788626684113112,0,-0.021522577897967115,0,0.021263586985002924,0,-0.021012135212097813,0,0.020767149642165764,0,
                  -0.02052907471942152,0,0.020296940019762573,0,-0.020071156753429097,0,0.019850844443845897,0,-0.019636384241486986,0,0.01942697491018363,0,-0.019222970354180382,0,0.019023639409863,0,-0.018829311234456036,0,
                  0.018639316851193865,0,-0.0184539628950705,0,0.018272635764088607,0,-0.018095621555770172,0,0.017922356134314317,0,-0.017753106834158233,0,0.017587353841065526,0,-0.01742534731602102,0,0.017266607270542855,0,
                  -0.0171113681191993,0,0.016959185756841244,0,-0.016810280135262496,0,0.016664239564119908,0,-0.01652127068935175,0,0.016380991174268528,0,-0.016243595403648357,0,0.016108727684773883,0,-0.015976571086375572,0,
                  0.015846794154317726,0,-0.015719569497848144,0,0.015594587760345138,0,-0.015472011869250384,0,0.015351552654715741,0,-0.015233364069636315,0,0.015117175421513142,0,-0.015003132332976848,0,0.014890981055920708,0,
                  -0.01478085947056775,0,0.014672529395365034,0,-0.014566121505337069,0,0.014461411944358672,0,-0.014358524673926053,0,0.014257249043017266,0,-0.014157702750251772,0,0.014059687336389341,0,-0.01396331465081806,0,
                  0.013868397507763253,0,-0.013775042287585302,0,0.01368307224420344,0,-0.013592588638885529,0,0.013503424406873217,0,-0.013415676012839663,0,0.013329185382363048,0,-0.01324404448111079,0,0.013160103594358576,0,
                  -0.013077450463709209,0,0.01299594315764607,0,-0.012915665448030557,0,0.012836482658740685,0,-0.012758474827420282,0,0.012681514049372955,0,-0.01260567684638583,0,0.012530841640771919,0,-0.012457081641134845,0,
                  0.012384281188129462,0,-0.012312510365487946,0,0.012241659055901143,0,-0.012171794393381537,0,0.012102811455691634,0,-0.012034774590200054,0,0.011967583749423295,0,-0.01190130064606918,0,0.011835829811322907,0,
                  -0.011771230465013025,0,0.011707411442980147,0,-0.011644429604560611,0,0.01158219783636913,0,-0.01152077076097909,0,0.011460065080283865,0,-0.011400133295833555,0,0.011340895706120898,0,-0.011282402800032896,0,
                  0.011224578269380255,0,-0.011167470691923559,0,0.011111006963628933,0,-0.011055233846352467,0]
    cdivs = np.r_[0,-0.5236021679762689,0,0.2668265375216003,0,-0.2138588885078355,0,0.16340716944222433,0,-0.14374195053979258,0,0.12163133468800147,0,-0.11105357014875379,0,
                    0.09842721661111303,0,-0.09170459942295543,0,0.0834513298300017,0,-0.07875028796410843,0,0.07289234289061199,0,-0.06939433853859696,0,0.0649989816301567,0,-0.06228004572814792,
                    0,0.058847383435008935,0,-0.05666448645565568,0,0.053901377240812835,0,-0.05210453059059276,0,0.049827167182899756,0,-0.04831844109332094,0,0.046405435469457405,0,
                    -0.04511798621091335,0,0.043485783765055346,0,-0.04237231561348309,0,0.040961424214400514,0,-0.039987463352061145,0,0.03875431301721787,0,-0.03789409582718294,0,
                    0.03680600751110985,0,-0.03603986530046248,0,0.03507182248231668,0,-0.03438446103197153,0,0.03351697073061856,0,-0.03289630524784267,0,0.03211394109412268,0,
                    -0.03155028715789527,0,0.030840672786318486,0,-0.030326171958804805,0,0.029679256397774162,0,-0.029207460388461105,0,0.02861499173794182,0,-0.02818055606417887,0,
                    0.027635692696476564,0,-0.02723414745470206,0,0.02673116641515587,0,-0.026358742318178426,0,0.025892817605850386,0,-0.025546312263805326,0,0.025113344132090574,0,
                    -0.024790018063714832,0,0.02438650009766833,0,-0.024083994995983213,0,0.02370690952195814,0,-0.02342318338034317,0,0.02306991837642265,0,-0.022803193530853912,0,
                    0.02247147603170117,0,-0.02222019719935423,0,0.021908039482276296,0,-0.021670839610340185,0,0.02137649537769073,0,-0.021152167648212632,0,0.020874096095838134,0,
                    -0.020661570822648693,0,0.02039840697983669,0,-0.020196732422908013,0,0.019947262517469026,0,-0.019755588856736252,0,0.019518729734978483,0,-0.019336295609425902,0,
                    0.019111077449793935,0,-0.01893719859250217,0,0.01872275031120504,0,-0.0185568099070849,0,0.01835234677685882,0,-0.018193787244196947,0,0.017998600342586924,0,
                    -0.017846916297602218,0,0.01766036347554594,0,-0.017515095684798327,0,0.01733659380481066,0,-0.017197323966401806,0,0.01702634220597082,0,-0.016892688429214008,0,
                    0.01672874248189291,0,-0.016600355358136323,0,0.01644300239437582,0,-0.016319561570176402,0,0.016168395843714257,0,-0.016049607022560097,0,0.015904256027457157,0,
                    -0.01578984833841616,0,0.015649969437506808,0,-0.015539693119151288,0,0.015404970577473106,0,-0.015298594933615451,0,0.015168737300918821,0,-0.015066048891426697,0,
                    0.014940786686540237,0,-0.014841587722104756,0,0.014720671379113953,0,-0.014624778293477044,0,0.01450797633566316,0,-0.014415218512692646,0,0.014302315925154532,0,
                    -0.014212534561407992,0,0.014103331337477609,0,-0.014016378423615156,0,0.013910688263690121,0,-0.013826425670414205,0,0.01372407481478928,0,-0.013642373470931154,0,
                    0.01354319965072664,0,-0.01346393880275017,0,0.013367790295159734,0,-0.013290856838775506,0,0.013197591614666049,0,-0.013122879490440335,0,0.013032364443896844,0,
                    -0.012959774089774532,0,0.012871884340492091,0,-0.012801322195043837,0,0.012715940455619235,0,-0.012647318506574705,0,0.012564334507729305,0,-0.012497569881025381,0,
                    0.012416879848629507,0,-0.012351894433767574,0,0.012273400612275798,0,-0.012210120720275086,0,0.012133730937814705,0,-0.012072086988482783,0,0.011997714259383131,0,
                    -0.011937640494992135,0,0.011865202656036968,0,-0.011806636878825431,0,0.011736056255911885,0,-0.011678939587118855,0,0.011610142689389442,0,-0.011554419347770203,0,
                    0.011487336586596513,0,-0.011432953684599083,0,0.01136751911508094,0,-0.011314426471053109,0,0.011250577553946607,0,-0.011198727518906802,0,0.011136404901112989,0,
                    -0.011085752198781471,0,0.011024899510718332]

    return [Fourier(*args) for args in [("Parabola",       "x-x²", lambda x: x*(1-x),              lambda n: 4*(1 - (-1)**n)/(n**3*π**3), (1/3, lambda n: -2*(1+(-1)**n)/(n*π)**2), 100),
                                        ("Cubic",    "5x³-7x²+2x", lambda x: 5*x**3-7*x**2+2*x,    lambda n: 4*(7 + 8*(-1)**n)/(n**3*π**3), (-1/6, lambda n: (60*(1-(-1)**n) + (6*(-1)**n-4)*(n*π)**2)/(n*π)**4), 100),
                                        ("sin(4πx)",   "sin(4πx)", lambda x: sin(4*π*x),           lambda n: n==4, (0, lambda n: 8*(1-(-1)**n)/((16-n**2+1e-200)*π)), 100),
                                        ("sin²(2πx)", "sin²(2πx)", lambda x: sin(2*π*x)**2,        lambda n: ((-1)**n - 1)/((n**3/16 - n + 1e-200)*π), (1,lambda n: -0.5*(n==4)), 100),
                                        ("Half circle", "√(x-x²)", lambda x: np.sqrt(x*(1-x)),     lambda n: jv(1,n*π/2)*sin(n*π/2)/n, (π/4, lambda n: jv(1,n*π/2)*cos(n*π/2)/n), 100),
                                        ("Shifted hat", "x θ(⅓-x) + (1-x)/2 θ(x-⅓)",               lambda x: (x<=w)*x + (x>w)*w*(1-x)/(1-w), lambda n: 2*sin(n*π*w)/((n*π)**2*(1-w)), 
                                                                                                   (w, lambda n: (2*(w*(1-(-1)**n) - 1 + cos(n*π*w)))/((n*π)**2*(1-w))), 100),
                                        ("Constant",          "1", lambda x: np.ones_like(x),      lambda n: 2*(1-(-1)**n)/(n*π), (2,lambda n: 0*n), 300),
                                        ("Sawtooth", "x - θ(x-½)", lambda x: x - (x>0.5),          lambda n: -2*cos(n*π/2)/(n*π), (0,lambda n: 2*((-1)**n-1+n*π*sin(n*π/2))/(n*π)**2), 300),
                                        ("Quarter circle", "√(1-x²)", lambda x: np.sqrt(1-x**2),   lambda n: struve(1,n*π)/n, (π/2, lambda n: jv(1,n*π)/n), 300),
                                        ("Divergence", "|x-½|^(-¼) - 2^¼", lambda x: abs(x-0.5)**(-1/4) - 2**(1/4), lambda n: adivs[n-1], (2**(1/4)*2/3, lambda n: cdivs[n-1]), 300),
                                       ]]

fouriers = make_fouriers()

In [4]:
# fourier plot
def make_fourier_plot(fouriers):
    labels  = [fou.label  for fou in fouriers]
    legends = [fou.legend for fou in fouriers]
    
    nmax = np.max([fou.nmax for fou in fouriers])
    λmin = 2/nmax
    N = int(5/λmin) + 1
    
    x = np.linspace(0,1,N)

    ys = [fou.f(x) for fou in fouriers]
    As = [fou.a(np.arange(1,nmax+1)) for fou in fouriers]

    source = ColumnDataSource(data=dict(x=x, y=ys[0], yS=As[0][0]*sin(π*x)))
    source_A = ColumnDataSource(data=dict(n=np.arange(1,nmax+1), a=As[0]))

    fig = figure(width=900,height=500)
    fig.line('x','y', source=source,line_width=5,color=colors[0],legend_label=f"f(x) = {legends[0]}")
    fig.line('x','yS',source=source,line_width=3,color=colors[1],legend_label="Sₙ(x) = Σₘⁿ aₘ sin kₘx")
    set_fig_style(fig,xmin=0,xmax=1,ymax=1.)
    del fig.y_range
    
    fig_A = figure(width=900,height=150)
    fig_A.vbar('n', width=0.8, top='a', source=source_A, color=colors[1],legend_label=f"aₙ")
    set_fig_style(fig_A,xlab='$$n$$',ylab=r'$$a_n$$',xmin=0.3,xmax=50.7,ymax=1.)
    del fig_A.y_range

    buttons = RadioButtonGroup(labels=labels, active=0)
    slider_n = Slider(start=1, end=len(As[0]), value=1, step=1, title="n")

    args = dict(source=source, source_A=source_A, slider_n=slider_n, buttons=buttons, ys=ys, As=As, legends=legends, legitem=fig.legend.items[0])
    slider_n.js_on_change('value', CustomJS(args=args,
                                            code="""const sin=Math.sin, cos=Math.cos, PI=Math.PI;
                                                    const yS = source.data["yS"];
                                                    const ii = buttons.active;
                                                    const nmax = slider_n.value;
                                                    const A = As[ii].slice(0,nmax);
                                                    source.data["x"].forEach((x,i) => {
                                                        yS[i] = A.reduce((y,a,nm1) => y + a*sin((nm1+1)*PI*x), 0.)
                                                    });
                                                    source.change.emit();
                                                    source_A.data["a"] = As[ii];
                                                    source_A.change.emit();
                                                    """))
    buttons.js_on_change('active', CustomJS(args=args,
                                            code="""const ii = buttons.active;
                                                    source.data["y"] = ys[ii];
                                                    legitem.label = `f(x) = ${legends[ii]}`;
                                                    slider_n.end = As[ii].length;
                                                    slider_n.value = 2; slider_n.value = 1;"""))
    layout = column(buttons,slider_n,fig,fig_A)
    show(layout)

make_fourier_plot(fouriers)

/var/folders/k6/1h87k8n54n54f6zxqf4w_k5m0000gn/T/ipykernel_15444/3583138035.py:57: RuntimeWarning: divide by zero encountered in power
  ("Divergence", "|x-½|^(-¼) - 2^¼", lambda x: abs(x-0.5)**(-1/4) - 2**(1/4), lambda n: adivs[n-1], (2**(1/4)*2/3, lambda n: cdivs[n-1]), 300),


# Different Fourier series
Here we show the Fourier series corresponding to Dirichlet ($y(x=0,t)=y(x=L,t)=0$), Neuman ($\frac{\partial y}{\partial x}(x=0,t) = \frac{\partial y}{\partial x}(x=L,t)=0$), and periodic ($y(x=0,t)=y(x=L,t)$ y $\frac{\partial y}{\partial x}(x=0,t)=\frac{\partial y}{\partial x}(x=L,t)$ boundary conditions. They are given by 
\begin{align}
S_n(x) &= \sum_{m=1}^n a_m \sin\left(\frac{m\pi}{L} x\right)\\
C_n(x) &= b_0 + \sum_{m=1}^{n-1} b_m \cos\left(\frac{m\pi}{L} x\right)\\
F_n(x) &= B_0 + \sum_{m=1}^{(n-1)/2} \left(A_{m} \sin\left(\frac{2m\pi}{L} x\right) + B_{m} \cos\left(\frac{2m\pi}{L} x\right) \right)
\end{align}

(The $(n-1)/2$ in $F_n$ is used as a sloppy way to indicate that we always use $n$ terms below, taking them in the order $B_0, A_1, B_1, A_2, \ldots$. E.g., for $n=2$, we include the $B_0$ and $A_1$ terms, but not $B_1$.)

For a given function $f(x)$ defined in the interval $x \in [0,L]$, the coefficients are obtained as 
\begin{align}
a_m &= \frac{2}{L} \int_0^L f(x) \sin\left(\frac{m\pi}{L} x\right) \mathrm{d}x\\
b_0 &= \frac{1}{L} \int_0^L f(x) \mathrm{d}x & b_{m>0} &= \frac{2}{L} \int_0^L f(x) \cos\left(\frac{m\pi}{L} x\right) \mathrm{d}x\\
A_m &= a_{2m} & B_m &= b_{2m}
\end{align}

The equalities for $A_m$ and $B_m$ use that these are the coefficients of a subset of the basis functions used in $S_n(x)$ and $C_n(x)$, respectively.

We note that the series $S_n(x)$ corresponds to an antisymmetric extension of the function $f(x)$ to the whole real axis with periodicity $2L$, i.e., $S_n(-x) = -S_n(x)$, $S_n(x+2L) = S_n(x)$, while $C_n$ is a symmetric extension with the same periodicity, $C_n(-x) = C_n(x)$, $C_n(x+2L) = C_n(x)$. Finally, $F_n(x)$ is an extension with periodicity $L$, i.e., $F_n(x+L) = F_n(x)$.

In [5]:
# Plot three different fourier series (Dirichlet, Neuman, periodic boundary conditions)
def make_fourier_plot_three_bases(fouriers):
    labels  = [fou.label  for fou in fouriers]
    legends = [fou.legend for fou in fouriers]
    
    nmax = np.max([fou.nmax for fou in fouriers])
    λmin = 2/nmax
    N = int(5/λmin) + 1
    
    x = np.linspace(0,1,N)
    xF = np.linspace(-0.5,1.5,2*N)

    ys = [fou.f(x) for fou in fouriers]
    As = [np.r_[0,        fou.a(   np.arange(1,fou.nmax+1))] for fou in fouriers]
    Cs = [np.r_[fou.c[0], fou.c[1](np.arange(1,fou.nmax+1))] for fou in fouriers]

    source_f = ColumnDataSource(data=dict(x=x, y=ys[0]))
    source_F = ColumnDataSource(data=dict(x=xF, yS=0*xF, yC=Cs[0][0]/2 + 0*xF, yP=Cs[0][0]/2 + 0*xF))

    fig = figure(width=900,height=500)
    fig.line('x','y',source=source_f,line_width=5,color=colors[0],legend_label=f"f(x) = {legends[0]}")
    fig.line('x','yS',source=source_F,line_width=3,color=colors[1],legend_label="Sₙ(x)")
    fig.line('x','yC',source=source_F,line_width=3,color=colors[2],legend_label="Cₙ(x)")
    fig.line('x','yP',source=source_F,line_width=3,color=colors[6],line_dash="dashed",legend_label="Fₙ(x)")
    
    set_fig_style(fig,xmin=-0.5,xmax=1.5,ymax=1.)
    del fig.y_range
    
    buttons = RadioButtonGroup(labels=labels, active=0)
    slider_n = Slider(start=0, end=len(As[0])-1, value=0, step=1, title="n")

    args = dict(source_f=source_f, source_F=source_F, slider_n=slider_n, buttons=buttons,
                ys=ys, As=As, Cs=Cs, legends=legends, legitem=fig.legend.items[0])
    slider_n.js_on_change('value', CustomJS(args=args,
                                            code="""const sin=Math.sin, cos=Math.cos, PI=Math.PI;
                                                    const yS = source_F.data["yS"], yC = source_F.data["yC"], yP = source_F.data["yP"];
                                                    const ii = buttons.active;
                                                    const nmax = slider_n.value;
                                                    const A = As[ii], C = Cs[ii];
                                                    source_F.data["x"].forEach((x,i) => {
                                                        yS[i] = A.slice(0,nmax+1).reduce((y,a,n) => y + a*sin(n*PI*x), 0.);
                                                        yC[i] = C.slice(1,nmax+1).reduce((y,c,nm1) => y + c*cos((nm1+1)*PI*x), C[0]/2);
                                                        // order const, sin, cos, sin, ...
                                                        yP[i] = C[0]/2;
                                                        for (var n=1; n<=nmax; n+=2) { yP[i] += A[n+1]*sin((n+1)*PI*x) }
                                                        for (var n=2; n<=nmax; n+=2) { yP[i] += C[n]*cos(n*PI*x) }
                                                    });
                                                    source_F.change.emit();"""))
    buttons.js_on_change('active', CustomJS(args=args,
                                            code="""const ii = buttons.active;
                                                    source_f.data["y"] = ys[ii];
                                                    legitem.label = `f(x) = ${legends[ii]}`;
                                                    slider_n.end = As[ii].length - 1;
                                                    slider_n.value = 1; slider_n.value = 0;
                                                    source_f.change.emit();"""))
    layout = column(buttons,slider_n,fig)
    show(layout)

make_fourier_plot_three_bases(fouriers)

/var/folders/k6/1h87k8n54n54f6zxqf4w_k5m0000gn/T/ipykernel_15444/3583138035.py:57: RuntimeWarning: divide by zero encountered in power
  ("Divergence", "|x-½|^(-¼) - 2^¼", lambda x: abs(x-0.5)**(-1/4) - 2**(1/4), lambda n: adivs[n-1], (2**(1/4)*2/3, lambda n: cdivs[n-1]), 300),
